In [49]:
import re
import pickle
import json
import pandas as pd

from library.common import Core
from pathlib import Path
cwd = Path.cwd()
prediction_horizon = 10
core = Core()
models = core.all_features

In [50]:
models_path = cwd.parent/'models'
models = list(models_path.glob('*.pkl'))

In [51]:
root_dir = cwd.parent
data_file= r'data/processed/params_forecast_params.json'
last_data_year_file = root_dir/data_file
file_path = last_data_year_file.as_posix()

In [52]:

with open(file_path, 'r') as datapoint:
    last_data_year = json.load(datapoint)

year_labels = [n for n in range(last_data_year.get('last_period') + 1,
                                last_data_year.get('last_period') + 1 +
                                prediction_horizon, 1)]

In [53]:
regex = r"-([^-]\w*[^-])-([^-]\w*[^\.])"
forecasts = []
for model in models:
    print('Processing:', model)
    fname = model.name
    matches = re.search(regex, fname)

    if matches:
        region = matches.group(1)
        feature = matches.group(2)

        with open(model.as_posix(), 'rb') as pkl:
            optimal = pickle.load(pkl)
            fc = zip(year_labels, optimal.predict(prediction_horizon))
            forecast_df = pd.DataFrame(fc)
            forecast_df.columns = ['year', 'forecasts']
            forecasts.append([region, feature, forecast_df.to_numpy()])

Processing: /Users/koonleong/Documents/UM/UMMADS/Courses - Current/SIADS697-Capstone/power_emissions/models/model-china-trade_openness.pkl
Processing: /Users/koonleong/Documents/UM/UMMADS/Courses - Current/SIADS697-Capstone/power_emissions/models/model-china-constant_gdp_per_capita.pkl
Processing: /Users/koonleong/Documents/UM/UMMADS/Courses - Current/SIADS697-Capstone/power_emissions/models/model-china-co2_emission_per_constant_gdp.pkl
Processing: /Users/koonleong/Documents/UM/UMMADS/Courses - Current/SIADS697-Capstone/power_emissions/models/model-usa-primary_energy_consumption_per_capita.pkl
Processing: /Users/koonleong/Documents/UM/UMMADS/Courses - Current/SIADS697-Capstone/power_emissions/models/model-usa-manufacturing_country_share_percent.pkl
Processing: /Users/koonleong/Documents/UM/UMMADS/Courses - Current/SIADS697-Capstone/power_emissions/models/model-usa-co2.pkl
Processing: /Users/koonleong/Documents/UM/UMMADS/Courses - Current/SIADS697-Capstone/power_emissions/models/model-c

In [55]:
fc_path = cwd.parent/r'data/processed/ts_forecast.json'
with open(fc_path, 'wb') as pkl:
    pickle.dump(forecasts, pkl)

print("Done with Forecasts")

Done with Forecasts
